In [43]:
from requests import get
import json
import pandas as pd

In [44]:
# hev rå data ud af tv2s og konverterede manuelt til en json
storkredse = json.load(open("raw_data/TV2/greaterConstituencies.json"))

In [45]:
# bemærkede at tv hentede svarende på den her måde
svar = {}
for storkreds in storkredse:
    print(storkreds['value'], storkreds['label'])
    res = get(f"https://election-api.services.tv2.dk/fv/fv22/candidatetest/answers/{storkreds['value']}")
    svar[storkreds['label']] = res.json()
#gem svarerne
json.dump(svar, open('raw_data/TV2/alle_svar.json', 'w'))

9000400 Bornholms Storkreds
9000600 Fyns Storkreds
9000200 Københavns Omegns Storkreds
9000100 Københavns Storkreds
9001000 Nordjyllands Storkreds
9000300 Nordsjællands Storkreds
9000500 Sjællands Storkreds
9000700 Sydjyllands Storkreds
9000900 Vestjyllands Storkreds
9000800 Østjyllands Storkreds


In [46]:
#svar = json.load(open('raw_data/TV2/alle_svar.json'))

In [47]:
# pt ignorere vi "regionale spørgsmål" fordi de i sagens natur kun er besvaret af et mindretal af kandidaterne...måske behøver jeg ikke gøre det...
dk_spg = ["1a", "1b", "2a", "2b", "3a", "3b", "4a", "4b", "5a", "5b", "6a", "6b", "7a", "7b", "8a", "8b", "9a", "9b", "10a", "10b", "11a", "11b", "12a", "12b"]

In [48]:
# smæk det hele pænt sammen

dk_svar = {}
dk_kandidater = {}
for storkreds, kandidater in svar.items():
    for kandidat in kandidater:
        dk_kandidater[kandidat['id']] = {'navn': kandidat['name'], 'storkreds': kandidat['areaName'], 'parti': kandidat['partyName'], 'alder': kandidat['age']}
        dk_svar[kandidat['id']] = {x: kandidat['answers'][x]['answer'] for x in dk_spg if x in kandidat['answers']}

In [49]:
dk_kandidater = pd.DataFrame(dk_kandidater).T
dk_svar = pd.DataFrame(dk_svar).T

In [177]:
# norm mellem 0 og 1
ma = dk_svar.max()
mi = dk_svar.min()
dk_svar = (dk_svar-mi)/(ma-mi)

In [178]:
q = dk_svar.copy()
q['parti'] = [dk_kandidater.loc[x, 'parti'] for x in dk_svar.index]
q['navn'] = [dk_kandidater.loc[x, 'navn'] for x in dk_svar.index]
q['storkreds'] = [dk_kandidater.loc[x, 'storkreds'] for x in dk_svar.index]
q['alder'] = [dk_kandidater.loc[x, 'alder'] for x in dk_svar.index]
q = q.reset_index()

In [179]:
# udfyld manglende svar med partiets gennemsnit
parti_linjen = q.groupby('parti')[dk_spg].mean()

def fald_ind(x):
    x.loc[x[x.isna()].index] = parti_linjen.loc[x.parti, x[x.isna()].index]
    return x
    
q = q.apply(fald_ind, axis=1)

In [85]:
# DR's data er...mere gemt...men jeg fandt en acceptabel måde at hive det ud på via scrapy
import pickle
import pandas as pd

In [97]:
objects = []
with (open("TakeTheDR/outputs.pickle", "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break

In [98]:
pd.DataFrame([[len(y) for y in x['kandidat']] for x in objects])[0].value_counts()

25    838
0     125
Name: 0, dtype: int64

In [99]:
# 838 har svaret.

In [100]:
# Lets remove them
objects = [x['kandidat'] for x in objects if x['kandidat'][0]]

In [101]:
sum(pd.Series([pd.DataFrame(x[0])['Answer'].sum() for x in objects])==0)
# hmm og 11 har enten svaret total nej til alt eller har ikke rigtigt svaret

11

In [102]:
objects = [x for x in objects if pd.DataFrame(x[0])['Answer'].sum() != 0]

In [103]:
len(objects)

827

In [104]:
beggetests = pd.Series([sum(q.navn == " ".join([x[1]['Firstname'], x[1]['LastName']])) for x in objects])
beggetests.mean()

0.8573155985489722

In [105]:
#ok 86% af folkenene findes i begge tests...

In [106]:
# Who the fuck tænker "det springer jeg lige over"....
# Lets find out =)

In [107]:
navne = pd.Series([" ".join([x[1]['Firstname'], x[1]['LastName']]) for x in objects])

In [108]:
navne[beggetests == 0]

8               Pavia Jacobsen
9               Martin Kjærulf
11     Chresten Heesgård Ibsen
16         Thor Clasen Jonasen
18          Rasmus Bøgh Vinter
                ...           
789        Karsten Jul Brædder
792               Rico Djernis
802       Allan Bilgaard Beier
806         Peter Zinckernagel
825                  Ida Auken
Length: 118, dtype: object

In [205]:
# Ida..tsk tsk
# Nå ..videre

In [159]:
bogstaver = json.load(open('various.json'))['bogstav_leg']

In [206]:
dr_data = []
for i, dd in enumerate(objects):
    person = {'navn': " ".join([dd[1]['Firstname'], dd[1]['LastName']]), 'parti': bogstaver[dd[1]['CurrentPartyCode']]}
    svar = pd.DataFrame(dd[0])[['QuestionID', 'Answer']].set_index('QuestionID').T/5
    svar.columns = svar.columns.astype('str')
    svar['navn'] = person['navn']
    svar['parti'] = person['parti']
    svar.index = [f'DR{i}']
    dr_data.append(svar)
dr_df = pd.concat(dr_data)

In [207]:
all_data = dr_df.merge(q, on=['navn', 'parti'])

In [208]:
all_data = all_data.drop('index', axis=1)

In [209]:
all_data

,530,531,533,534,535,537,538,540,541,543,...,9a,9b,10a,10b,11a,11b,12a,12b,storkreds,alder
0,1.0,1.0,0.2,1.0,0.2,0.2,1.0,0.2,1.0,0.2,...,0.00,0.50,0.50,1.00,0.75,0.25,0.50,0.50,Bornholms Storkreds,34
1,0.4,0.4,0.8,0.2,1.0,0.8,0.2,0.8,0.2,0.8,...,0.75,0.50,0.75,0.75,0.00,1.00,0.75,0.75,Bornholms Storkreds,56
2,1.0,0.8,0.2,0.2,0.8,0.2,0.8,0.4,0.8,0.4,...,0.25,0.00,0.50,1.00,0.50,0.25,0.75,0.00,Bornholms Storkreds,37
3,0.8,0.2,0.8,0.2,1.0,0.8,0.2,1.0,0.4,0.8,...,0.75,0.25,0.75,0.25,0.25,1.00,0.75,0.25,Bornholms Storkreds,27
4,0.4,0.2,0.4,0.2,1.0,0.8,0.2,1.0,0.2,0.8,...,0.75,0.25,0.75,1.00,0.00,0.75,0.75,0.75,Bornholms Storkreds,69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
704,0.8,0.8,0.8,0.4,0.8,0.4,0.4,0.4,0.8,0.4,...,0.25,0.50,0.50,0.75,0.75,0.25,0.75,0.50,Københavns Storkreds,43
705,0.8,0.8,0.8,0.2,1.0,0.4,0.8,0.4,0.4,0.8,...,0.75,0.50,0.25,1.00,1.00,0.00,0.75,0.50,Københavns Storkreds,78
706,0.8,1.0,0.8,0.2,0.8,0.2,0.8,0.4,0.8,0.4,...,0.00,0.75,0.50,1.00,0.75,0.00,0.50,0.00,Københavns Storkreds,56
707,0.8,0.8,0.8,0.2,1.0,0.4,0.8,0.4,0.4,0.8,...,0.25,0.50,0.25,1.00,0.75,0.00,0.50,0.50,Københavns Storkreds,32


In [210]:
all_data.to_feather("2022_Lasse_data.feather")